# 숫자형 -> 숫자형
- 시각화 : scatter 
    - sns.scatterplot(x = '변수', y = '변수', data = df)
    - sns.regplot(x = '변수', y = '변수', data = df)
- 수치화 : 상관분석
    - NaN 확인 = df.isna().sum()
    - NaN 제거 = temp = df.loc[tdf['변수'].notnull()] 
    - spst.pearsonr(df['변수'], df['변수']) -> (상관계수, p-value) |상관계수| 1에 가까울 수록, p-value < 0.05 관계가 있다.

# 숫자형 -> 범주형
- 시각화 :
    - Boxplot : 
    - Histogram : sns.histplot(x='feature 변수', data = df, hue = '타겟변수')
    - Density plot :
        * 두가지 방법이 있습니다.
        * ① kdeplot(   , hue = 'Survived') 
            * 생존여부의 비율이 유지된 채로 그려짐
            * 두 그래프의 아래 면적의 합이 1
            - sns.kdeplot(x='Age', data = titanic, hue ='Survived')
        * ② kdeplot(   , hue = 'Survived', common_norm = False)
            * 생존여부 각각 아래 면적의 합이 1인 그래프
            - sns.kdeplot(x='Age', data = titanic, hue ='Survived', common_norm = False)
        * ③ kdeplot(   , hue = 'Survived', multiple = 'fill')
            * 나이에 따라 생존여부 **비율**을 비교해볼 수 있음. (양의 비교가 아닌 비율!)
            - sns.kdeplot(x='Age', data = titanic, hue ='Survived', multiple = 'fill')
            - plt.axhline(titanic['Survived'].mean(), color = 'r')


# 범주형 -> 범주형
- 시각화 :
    - 교차표 : 
        - # 두 범주별 빈도수를 교차표로 만들어 봅시다.
        - pd.crosstab(df['변수'], df['변수']) normalize = columns, index, all
        - <img src='https://github.com/DA4BAM/image/blob/main/crosstab.png?raw=true' width=700/>
    - Barplot : 
    - Mosaic : mosaic(dataframe, [ feature, target]), plt.axhline(1- df['target'].mean(), color = 'r')
- 수치화 : 
    - 카이 제곱 검정 : 카이제곱 통계량이 자유도의 2배보다 크면 관계가 있다고 봄(유의미)
        1. 교차표 집계 : table = pd.crosstab(df['변수'], df['변수']) ->  Normalize 옵션을 사용하면 안됨.
        2. 카이제곱검정 : spst.chi2_contingency(table)
        3. 결과 확인 : 카이제곱 검정 결과 순서대로① 카이제곱 통계량
② P-value
③ 자유도 : Pclass 자유도(3-1) * Survived 자유도(2-1)
④ 기대빈도 : 계산된 값

# 범주형 -> 숫자형
- 시각화 : 
    - barplot(sns.barplot) : sns.barplot(x = '변수', y = '변수' data = df)
- 수치화 : 
    - NaN 확인 = df.isna().sum()
    - NaN 제거 = temp = df.loc[tdf['변수'].notnull()] 
    - t-test(범주 2개) : t값이 -2보다 작거나 2보다 크면 차이가 있다고 봄
        - 두 그룹으로 데이터 저장 -> a = df.loc[df['feature변수'] == df['feature변수'],unique(), 'target변수'] => unique()를 순서대로 넣어줌
        - spst.ttest_ind(a, b)
    - 분산 분석(ANOVA)(범주 3개 이상) :(집단 간 분산)/(집단 내 분산) = (전체 평균 − 각 집단 평균)/(각 집단의 평균 − 개별 값)
        - 각 그룹간 차이가 있는지 알려줌 차이가 있으면 사후분석 즉 따로 때내서 t-test함
        - 값이 대략 2~3 이상이면 차이가 있다고 판단합니다.
        - 다 동일하고 spst.
        - 범주의 종류가 너무 많으면 pd.cut()함수를 이용하여 범주형 변수로 변환하여 분석할 수도 있다.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as spst
import matplotlib.pyplot as plt
import seaborn as sns

# info : 범주 -> target : 숫자
def draw_barplot(x:str, y:str, data:pd.DataFrame):
    plt.figure(figsize = (10,4))
    sns.barplot(x=x, y=y, data=data)
    plt.grid()
    plt.xticks(rotation = 35) 
    plt.show()
    test = list()

    if data[x].isna().sum():
        data = data.loc[data[x].notnull(), :]

    if len(data[x].unique()) > 2:
        for u in data[x].unique():
            test.append(data.loc[data[x] == u, y])
        result = spst.f_oneway(*test)
        return print('F-score : {}, P-value : {}'.format(result[0], result[1]))
    else:
        for u in data[x].unique():
            test.append(data.loc[data[x] == u, y])
        result = spst.ttest_ind(*test)
        return print('T-score : {}, P-value : {}'.format(result[0], result[1]))
    
    
# info : 숫자 -> target : 숫자
def draw_scatter(x:str, y:str, data=pd.DataFrame):

    sns.jointplot(x=x, y=y, data=data)
    plt.show()
    
    sns.regplot(x=x, y=y, data=data)
    plt.show()

    result = spst.pearsonr(data[x],data[y])

    return print('Pearson : {}, p-value : {}'.format(result[0], result[1]))

In [1]:
# 단변량 숫자
def draw_one_num(data, var, bins):
    # 기초 통계량
    display(data[[var]].describe().T)
    
    # 시각화
    plt.figure(figsize = (8, 10))
    plt.subplot(2, 1, 1)
    sns.histplot(x = var, data = data, bins = bins, kde = True)
    
    plt.subplot(2, 1, 2)
    sns.boxplot(x = var, data = data)
    plt.grid()
    plt.show()